In [55]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

#kaggle.com/code/kritanjalijain/apriori-algorithm-hotel-management-system

In [56]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [57]:
# load data:
df_ht = pd.read_csv("hotel_bookings.csv")

In [77]:
# Replace missing values:
# agent: If no agency is given, booking was most likely made without one.
# company: If none given, it was most likely private.
# rest schould be self-explanatory.
nan_replacements = {"children:": 0.0,"country": "Unknown", "agent": 0, "company": 0}
df_clean = df_ht.fillna(nan_replacements)

# "meal" contains values "Undefined", which is equal to SC.
df_clean["meal"].replace("Undefined", "SC", inplace=True)

# Some rows contain entreis with 0 adults, 0 children and 0 babies. 
# I'm dropping these entries with no guests.
zero_guests = list(df_clean.loc[df_clean["adults"]
                   + df_clean["children"]
                   + df_clean["babies"]==0].index)
df_clean.drop(df_clean.index[zero_guests], inplace=True)

df_clean.head(5)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,0.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,0.0,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,0.0,0,Transient,98.0,0,1,Check-Out,2015-07-03


In [59]:
portugal_df = df_clean[df_clean["country"] == 'PRT']
france_df = df_clean[df_clean["country"] == 'FRA']
england_df = df_clean[df_clean["country"] == 'GBR']
germany_df = df_clean[df_clean["country"] == 'DEU']
italy_df = df_clean[df_clean["country"] == 'ITA']

portugal_df.head(10)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.00,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,0.0,0.0,0,Transient,0.00,0,0,Check-Out,2015-07-01
6,Resort Hotel,0,0,2015,July,27,1,0,2,2,...,No Deposit,0.0,0.0,0,Transient,107.00,0,0,Check-Out,2015-07-03
7,Resort Hotel,0,9,2015,July,27,1,0,2,2,...,No Deposit,303.0,0.0,0,Transient,103.00,0,1,Check-Out,2015-07-03
8,Resort Hotel,1,85,2015,July,27,1,0,3,2,...,No Deposit,240.0,0.0,0,Transient,82.00,0,1,Canceled,2015-05-06
9,Resort Hotel,1,75,2015,July,27,1,0,3,2,...,No Deposit,15.0,0.0,0,Transient,105.50,0,0,Canceled,2015-04-22
10,Resort Hotel,1,23,2015,July,27,1,0,4,2,...,No Deposit,240.0,0.0,0,Transient,123.00,0,0,Canceled,2015-06-23
11,Resort Hotel,0,35,2015,July,27,1,0,4,2,...,No Deposit,240.0,0.0,0,Transient,145.00,0,0,Check-Out,2015-07-05
14,Resort Hotel,0,37,2015,July,27,1,0,4,2,...,No Deposit,241.0,0.0,0,Transient,94.71,0,0,Check-Out,2015-07-05
16,Resort Hotel,0,37,2015,July,27,1,0,4,2,...,No Deposit,8.0,0.0,0,Contract,97.50,0,0,Check-Out,2015-07-05


In [60]:
clean_france_df = france_df[['stays_in_weekend_nights',
       'stays_in_week_nights',  'children', 'babies', 'previous_cancellations',
       'previous_bookings_not_canceled',  'booking_changes', 
       'required_car_parking_spaces', 'total_of_special_requests']]
#france_df.dropna(axis=0, subset =)
clean_france_df.head(10)

,stays_in_weekend_nights,stays_in_week_nights,children,babies,previous_cancellations,previous_bookings_not_canceled,booking_changes,required_car_parking_spaces,total_of_special_requests
18,0,1,0.0,0,0,0,0,0,0
80,2,7,0.0,0,0,0,0,0,0
101,2,1,0.0,0,0,0,0,1,0
102,2,4,2.0,0,0,0,0,0,2
127,1,1,0.0,0,0,0,0,1,1
129,1,1,0.0,0,0,0,0,0,2
130,1,1,0.0,0,0,0,0,0,1
141,2,0,0.0,0,0,0,0,1,0
189,3,8,0.0,0,0,0,1,0,0
242,2,5,0.0,0,0,0,0,0,0


In [61]:
# Defining the hot encoding function to make the data suitable
# for the concerned libraries
def hot_encode(x):
    if(x<= 0):
        return 0
    if(x>= 1):
        return 1

# Encoding the datasets
clean_france_df_encoded = clean_france_df.applymap(hot_encode)
clean_france_df = clean_france_df_encoded.copy()

In [76]:
# Building the model
frq_items = apriori(clean_france_df, min_support = 0.05, use_colnames = True)

# Collecting the inferred rules in a dataframe
rules = association_rules(frq_items, metric ="lift", min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules.head(20)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
25,"(children, total_of_special_requests)",(stays_in_week_nights),0.051534,0.953562,0.050380,0.977612,1.025221,0.001239,2.074224
4,(children),(stays_in_week_nights),0.082492,0.953562,0.080473,0.975524,1.023032,0.001812,1.897318
11,"(children, stays_in_weekend_nights)",(stays_in_week_nights),0.053649,0.953562,0.051630,0.962366,1.009232,0.000472,1.233920
30,"(booking_changes, total_of_special_requests)",(stays_in_week_nights),0.078839,0.953562,0.075474,0.957317,1.003938,0.000296,1.087972
7,(total_of_special_requests),(stays_in_week_nights),0.528026,0.953562,0.505048,0.956482,1.003062,0.001542,1.067099
0,(children),(stays_in_weekend_nights),0.082492,0.619652,0.053649,0.650350,1.049540,0.002532,1.087795
10,"(children, stays_in_week_nights)",(stays_in_weekend_nights),0.080473,0.619652,0.051630,0.641577,1.035383,0.001764,1.061171
20,"(booking_changes, total_of_special_requests)",(stays_in_weekend_nights),0.078839,0.619652,0.050284,0.637805,1.029295,0.001431,1.050119
2,(total_of_special_requests),(stays_in_weekend_nights),0.528026,0.619652,0.336506,0.637291,1.028465,0.009314,1.048630
24,"(children, stays_in_week_nights)",(total_of_special_requests),0.080473,0.528026,0.050380,0.626045,1.185633,0.007888,1.262115


In [ ]:
# for i in range(0,len(dataset)):
#     transactions.append({str(dataset.values[i,j]) for j in range(0,len(dataset.columns)) if str(dataset.values[i,j]) != 'nan'})

# print('Transactions:\n')
# for transaction in transactions:
#     print(transaction)

1. customers travel and stay with children have more special requests
2. customers who travel and stay during weekdays have more special requests

In [64]:
# def init_candidates(transactions):
#     candidates = dict()
#     for transaction in transactions:
#         for item in transaction:
#             itemset = set()
#             itemset.add(item)
#             itemset = frozenset(itemset)
#             if itemset not in candidates:
#                 candidates[itemset] = 1
#             else:
#                 candidates[itemset] += 1
                
#     return candidates


# def prune_candidates(candidates,support):
#     fp = dict()
#     for key in candidates:
#         if candidates[key] >= support:
#             fp[key] = candidates[key]
    
#     return fp


# def get_count(transactions,itemset):
#     itemset = set(itemset)
#     count = 0
#     for transaction in transactions:
#         if itemset.issubset(transaction):
#             count += 1
            
#     return count


# def get_candidates(fp,transactions,fp_length):
#     candidates = dict()
#     for key1 in fp:
#         for key2 in fp:
#             if key2 != key1:
#                 itemset1 = set(key1)
#                 itemset2 = set(key2)
#                 itemset = itemset1.union(itemset2)
#                 itemset = frozenset(itemset)
#                 if itemset not in candidates and len(itemset) == fp_length:
#                     candidates[itemset] = get_count(transactions,itemset)
    
#     return candidates


# def get_itemset_length(candidates):
#     itemset_len = 0
#     for key in candidates:
#         itemset_len = len(key)
#         break
    
#     return itemset_len


# import itertools

# def get_rules(fp):
#     rules = []
#     for itemset in fp:
#         itemset = set(itemset)
#         for length in range(1,len(itemset)):
#             subsets = set(itertools.combinations(itemset, length))
#             for subset in subsets:
#                 lhs = set(subset)
#                 rhs = itemset.difference(lhs)
#                 rules.append([lhs,rhs])
    
#     return rules


# def prune_rules(rules,confidence,transactions):
#     fp = []
#     for rule in rules:
#         lhs = rule[0]
#         rhs = rule[1]
#         lhs_rhs = lhs.union(rhs)
#         lhs_count = get_count(transactions,lhs)
#         lhs_rhs_count = get_count(transactions,lhs_rhs)
#         conf = lhs_rhs_count/lhs_count
#         if conf >= confidence:
#             support = lhs_rhs_count/len(transactions)
#             fp.append([lhs,rhs,support,conf])
    
#     return fp

# def display(candidates,fp,length):
#     print()
#     print("****** {}-Frequent Candidates ******".format(length))
#     for key in candidates:
#         print(set(key),' : ',candidates[key])
#     print()
#     print("****** {}-Frequent Itemlist ******".format(length))
#     for key in fp:
#         print(set(key),' : ',fp[key])
# def apriori(transactions,support,confidence):
#     support = support/100 * len(transactions)
#     confidence = confidence/100
#     candidates = init_candidates(transactions)
#     old_fp = prune_candidates(candidates,support)
#     final_fp = dict()
#     display(candidates,old_fp,1)
    
#     while True:
#         new_fp_length = get_itemset_length(candidates) + 1
#         candidates = get_candidates(old_fp,transactions,new_fp_length)
#         new_fp = prune_candidates(candidates,support)
#         display(candidates,new_fp,new_fp_length)
#         if len(new_fp)<1:
#             final_fp = old_fp
#             break
#         else:
#             old_fp = new_fp
    
#     rules = get_rules(final_fp)
#     fp = prune_rules(rules,confidence,transactions)
#     return fp
# support = float(input('Enter support percentage : '))
# confidence = float(input('Enter confidence percentage : '))
# fp = apriori(transactions,support,confidence)